# Atlas de l'expérience `keyexp`

In [ ]:
filename = 'resultat.nc'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import cartopy.crs as ccrs
from netCDF4 import Dataset

%matplotlib inline

data = Dataset(filename)

| Variable | Notation formelle usuelle | Nom dans `resultat.nc` | Unité | 
|:---------|:-----------------|:-----------------------|:------|
| Température de surface | $T_s(x,y,t)$ | `tsurf` | K |
| Pression de surface | $p_s(x,y,t)$ | `ps` | Pa |
| Pression atmosphérique | $p(x,y,z,t)$ | `p` | Pa |
| Température de l'atmosphère | $T(x,y,z,t)$ | `temp` | K |
| Température potentielle | $\theta(x,y,z,t)$ | `teta` | K |
| Vent zonal | $u(x,y,z,t)$ | `u` | m s$^{-1}$ |
| Vent méridien | $v(x,y,z,t)$ | `v` | m s$^{-1}$ |
| Vent vertical | $w(x,y,z,t)$ | `w` | m s$^{-1}$ |
| Albédo | $A_b(x,y,t)$ | `ALB` | SU |
| Flux solaire incident au sommet | $F_{vis}^{\downarrow_{TOA}}(x,y,t)$ | `ISR` | W m$^{-2}$ |
| Flux solaire absorbé par la surface | $F_{vis}^{\downarrow_{Surf}}(x,y,t)$ | `ASR` | W m$^{-2}$ |
| Flux infrarouge émis au sommet | $F_{IR}^{\uparrow_{TOA}}(x,y,t)$ | `OLR` | W m$^{-2}$ |
| Taux de chauffage SW | $Q_{rad}^{SW}(x,y,z,t)$ | `zdtsw` | K s$^{-1}$ |
| Taux de chauffage LW | $Q_{rad}^{LW}(x,y,z,t)$ | `zdtlw` | K s$^{-1}$ |
| Taux de chauffage total | $Q_{rad}(x,y,z,t)$ | `dtrad` | K s$^{-1}$ |
| RMM en vapeur d'eau | $q_{vap}(x,y,z,t)$ | `h2o_vap` | kg kg$^{-1}$ |
| RMM en glace d'eau | $q_{glace}(x,y,z,t)$ | `h2o_ice` | kg kg$^{-1}$ |
| Masse de glace au sol par m$^2$ | $m_{glace}(x,y,t)$ | `h2o_ice_surf` | kg m$^{-2}$ |

Quelques remarques : 
+ SW signifie shortwave, donc correspond au domaine des courtes longueurs d'onde (appelé aussi domaine "visible");
+ De la même façon LW signifie longwave, et correspond au domaine des grandes longueurs d'onde ou domaine infrarouge;
+ RMM signifie rapport de mélange **massique**, en kg de l'espèce considérée par kg d'air;
+ Le vent $\vec{v}$ s'écrit suivant les trois directions de l'espace $\vec{v}=u\ \hat{x}+v\ \hat{y}+w\ \hat{z}$.


In [ ]:
longitude=data.variables['longitude'][:]
latitude=data.variables['latitude'][:]
altitude=data.variables['altitude'][:]
Time = data.variables['Time'][:]
Ls = data.variables['Ls'][:]

$L_s$ est la longitude solaire, qui sert à désigner la position de la planète sur son orbite. Cumul des années pour les représentations graphiques :

In [ ]:
dafirst = Time[0]
daint = Time[1] - dafirst
dalast = dafirst + (len(Time)-1)*daint
year = 0.
add = np.linspace(dafirst,dalast,num=len(Time)) ; add[0] = 0.
for iii in range(1,len(Ls)):
    if Ls[iii] - Ls[iii-1] < 0: year = year+1.
    add[iii] = year*360.
Ls_true = add + Ls

## Topographie

In [ ]:
grav=9.81 # acceleration gravitationnelle
phis=data.variables['phisinit'][:]

dataplt = phis/grav

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(0,4e3,41)
plt.contourf(longitude, latitude, dataplt, clevs, transform=ccrs.PlateCarree(),cmap="terrain")
plt.title('Topographie', size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'mètres',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()

## Cycle saisonnier de la température

In [ ]:
tsurf=data.variables['tsurf'][:,:,:]
fig, ax = plt.subplots(figsize=(12,8))

ilat1 = 4
ilat2 = 5
ilat3 = 6
ilat4 = 7

# Using set_dashes() to modify dashing of an existing line
ts1, = ax.plot(Time, tsurf[:,ilat1,4] - 273.15, label='Latitude '+str(latitude[ilat1]))
ts2, = ax.plot(Time, tsurf[:,ilat2,4] - 273.15, label='Latitude '+str(latitude[ilat2]))
ts3, = ax.plot(Time, tsurf[:,ilat3,4] - 273.15, label='Latitude '+str(latitude[ilat3]))
ts4, = ax.plot(Time, tsurf[:,ilat4,4] - 273.15, label='Latitude '+str(latitude[ilat4]))
ax.set_ylim([-100,60])
ax.set_xlim([0,2000])
ax.set_xlabel('Solar longitude Ls')
ax.set_ylabel('Température de surface (°C)')
ax.grid()
ax.legend()

## Température moyenne globale

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

area = data.variables['aire'][:]
totarea = np.sum(np.sum(area[:,:],axis=0),axis=0)
tsmeanarea = np.mean(np.mean(tsurf[:,:,:]*area,axis=1),axis=1)/totarea*area.size
ts1, = ax.plot(Ls_true, tsmeanarea)
ax.set_xlabel('Solar longitude Ls')
ax.set_ylabel('Température de surface (K)')
ax.set_title("Evolution de la température de surface sur l'ensemble du globe")
ax.grid()
ax.legend()

## Bilan énergétique

Quatre figures donnant en fonction du temps $L_s$ et de la latitude :
1. le flux solaire incident au sommet $F_{vis}^{\downarrow_{TOA}}$,
2. le flux solaire absorbé par la surface $F_{vis}^{\downarrow_{Surf}}$,
3. $F_{IR}^{\uparrow_{TOA}}$ le flux infrarouge émis au sommet,
4. la température de surface $T_s$.

In [ ]:
ISR=data.variables['ISR'][:,:,:]
ASR=data.variables['ASR'][:,:,:]
OLR=data.variables['OLR'][:,:,:]
ncontour = 20

fig, axs = plt.subplots(2, 2,figsize=(12,10))
im1 = axs[0, 0].contourf(Ls_true,latitude,np.mean(ISR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
#axs[0, 0].cbar()
axs[0, 0].set_ylim([-90,90])
axs[0, 0].set_title('Incoming Solar Radiation')
fig.colorbar(im1,ax=axs[0, 0])

im2 = axs[0, 1].contourf(Ls_true,latitude,np.mean(ASR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
axs[0, 1].set_title('Absorbed Solar Radiation')
fig.colorbar(im2,ax=axs[0, 1])

im3 = axs[1, 0].contourf(Ls_true,latitude,np.mean(OLR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
axs[1, 0].set_title('OLR')
fig.colorbar(im3,ax=axs[1, 0])


im4 = axs[1, 1].contourf(Ls_true,latitude,(np.mean(tsurf[:,:,:],axis=2)-273).transpose(),
                         ncontour,cmap='bwr',vmin= -90,vmax=60)
axs[1, 1].set_title('Tsurf')
fig.colorbar(im4,ax=axs[1, 1])

for ax in axs.flat:
    ax.set(xlabel='Ls', ylabel='Latitude')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

## Graphiques à une longitude solaire $L_s$ donnée

### Choix du $L_s$

In [ ]:
# Paramètres utilisateurs -----------------------------------------
year_user = 4 # année de simulation à regarder
Ls_user = 270. # longitude solaire choisie

# Code ------------------------------------------------------------
Ls_true_user = year_user*360. + Ls_user

Ls_ind = np.where(abs(Ls_true-Ls_true_user)==
                  abs(Ls_true-Ls_true_user).min())[0]
print("La valeur la plus proche trouvée est Ls = "
      + str(Ls_true[Ls_ind]-year_user*360.)
      + " pour l'année " + str(year_user))
# -----------------------------------------------------------------

## Structure thermique

In [ ]:
# Paramètres utilisateurs -----------------------------------------
tempmin = 180.
tempmax = 360.
ncontour = 60

# Code ------------------------------------------------------------
temp = data.variables['temp'][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=latitude
yplt = np.squeeze(np.mean(np.mean(p[Ls_ind,:,:,:],axis=3),axis=2))
dataplt = np.squeeze(np.mean(temp[Ls_ind,:,:,:],axis=3))
im = ax.contourf(xplt,yplt,dataplt,ncontour,cmap='jet',vmax=tempmax, vmin=tempmin)
#axs[0, 0].cbar()
#ax1.set_ylim([-90,90])
ax.set_title('Temperature')
ax.set_yscale('log')
ax.set(xlabel='Latitude', ylabel='p')
ax.invert_yaxis()
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------